In [1]:
%pylab nbagg
import torch
import numpy as np
from lib.Simulator.Wrapper import *
from lib.utils import *
from numpy import array as ar
from copy import deepcopy as dcp
import numpy as np
from matplotlib import rc
import matplotlib as mpl
import multiprocessing
from multiprocessing import Pool
from lib.Simulator.connectivity_paths import connectivity_source_file
rc('text',usetex = True)
mpl.rcParams['font.size'] = 10
torch.set_printoptions(sci_mode=True)
try:
    multiprocessing.set_start_method('spawn')
except:
    pass
def paral_sim(s, ind,processes = 1):
    if processes > 11:
        processes = 11
    if __name__ == '__main__':
        with Pool(processes=processes) as pool:
            results = pool.map_async(s.run_simulation_long,ind).get(timeout=600000)                
        pool.close()
        pool.join()
    return results
sc = connectivity.Connectivity.from_file(connectivity_source_file)
SC_scaled = sc.weights* ((np.ones((68,68)) - np.eye(68)))
SC_scaled /= np.max(SC_scaled)
par_val_def = torch.cat([theta_initial.view(1,-1) for i in range(68)], dim = 0)
par_val_def[45,5] = 8
par_val_def_denoise = torch.cat([theta_initial.view(1,-1) for i in range(68)], dim = 0)
par_val_def_denoise[45,5] = 8
par_val_def_denoise[:,-1] = 0
par_val_def_denoise[:,-2] = 0
print(par_val_def.shape)

Populating the interactive namespace from numpy and matplotlib
torch.Size([68, 24])


In [2]:
wrapper = Wrapper()
par_vals = [[1e-4*i,10*j,f'Coupled{(i-1)*(4)+(j-1)} (a:{str(1e-4*i)[:6]}, K: {10*j})'] for i in range(1,7) for j in range(1,6)]
#a_vals.reverse()
wrapper.add_sim(par_val_def_denoise,file = 'empty', a_val = 0, speed = 50)
for i in range(len(par_vals)):
    wrapper.add_sim(par_val_def_denoise, a_val = par_vals[i][0], speed = par_vals[i][1]) 

In [ ]:
results = paral_sim(wrapper, [i for i in range(len(wrapper.simulations))],processes = len(wrapper.simulations))
psd = torch.cat([results[i][1][:,4:100].view((1,68,-1)) for i in range(len(results))],dim=0)

In [ ]:
num_sims = len(wrapper.simulations)
connectome = np.zeros((num_sims,68,68))
for i in range(num_sims):
    connectome[i] =  np.corrcoef(results[i][0])*(np.ones((68,68)) - np.eye(68))
print(connectome.shape)
conn_titles = ['Uncoupled']
for i in range(len(par_vals)):
    conn_titles.append(par_vals[i][2])
figure(figsize=(17,num_sims))
for i in range(num_sims):
    subplot(int(num_sims/4)+1,4,i+1)
    imshow(abs(connectome[i]))
    colorbar()
    title(conn_titles[i])
subplot(int(num_sims/4)+1,4,i+2)
imshow(SC_scaled)
colorbar()
title('SC')
tight_layout()

In [ ]:
PSD_show = True
plot_psd = [0,1,-1]
if PSD_show:
    freqs = [0.5*i for i in range(4,100)]
    figure(figsize=(16,60))
    counter = 0
    for i in range(68):
        counter+=1
        subplot(17,4,counter)
        plot(freqs,psd[plot_psd[0],i,:96],label = f'Uncoupled',linestyle = '-')
        plot(freqs,psd[plot_psd[1],i,:96],label = f'Coupled {plot_psd[1]}',linestyle = '-')
        plot(freqs,psd[plot_psd[2],i,:96],label = f'Coupled {plot_psd[2]}',linestyle = '-')
        #xlim([1,40])
        legend(loc = 'upper right')
        title(f"Region: {i}")
        grid(linestyle = '--')
    tight_layout()
    show()

In [ ]:
ts_show = True
plot_ts = [0,1,-1]
if ts_show:
    t = [5e-4*j for j in range(4000,6000)]
    figure(figsize=(16,15))
    for i in range(3):
        subplot(3,1,i+1)
        for j in range(68):
            plot(t,results[plot_ts[i]][0][j,4000:6000])
        title(f"Simulation: {plot_ts[i]}")
    
        grid(linestyle = '--')
    tight_layout()
    show()

In [ ]:
wrapper = Wrapper()
par_vals = [[1e-4*i,10*j,f'Coupled{(i-1)*(4)+(j-1)} (a:{str(1e-4*i)[:6]}, K: {10*j})'] for i in range(1,7) for j in range(1,6)]
#a_vals.reverse()
wrapper.add_sim(par_val_def,file = 'empty', a_val = 0, speed = 50)
for i in range(len(par_vals)):
    wrapper.add_sim(par_val_def, a_val = par_vals[i][0], speed = par_vals[i][1]) 

In [ ]:
results = paral_sim(wrapper, [i for i in range(len(wrapper.simulations))],processes = len(wrapper.simulations))
psd = torch.cat([results[i][1][:,4:100].view((1,68,-1)) for i in range(len(results))],dim=0)

In [ ]:
num_sims = len(wrapper.simulations)
connectome = np.zeros((num_sims,68,68))
for i in range(num_sims):
    connectome[i] =  np.corrcoef(results[i][0])*(np.ones((68,68)) - np.eye(68))
print(connectome.shape)
conn_titles = ['Uncoupled']
for i in range(len(par_vals)):
    conn_titles.append(par_vals[i][2])
figure(figsize=(17,num_sims))
for i in range(num_sims):
    subplot(int(num_sims/4)+1,4,i+1)
    imshow(abs(connectome[i]))
    colorbar()
    title(conn_titles[i])
subplot(int(num_sims/4)+1,4,i+2)
imshow(SC_scaled)
colorbar()
title('SC')
tight_layout()

In [ ]:
num_sims = len(wrapper.simulations)
eigen_dec_connectome = []
for c in connectome:
    val,vect = np.linalg.eig(c)
    val[3:] *= 0
    lambda_diag = np.diag(val)
    reconstructed_conn = vect @ lambda_diag @ vect.T
    eigen_dec_connectome.append(reconstructed_conn*(np.ones((68,68)) - np.eye(68)))
print(eigen_dec_connectome[0].shape)
#inter_conns = 

In [ ]:
figure(figsize=(17,num_sims))
for i in range(num_sims):
    subplot(int(num_sims/4)+1,4,i+1)
    #imshow(eigen_dec_connectome[i])
    imshow(abs(eigen_dec_connectome[i]))
    colorbar()
    title(conn_titles[i])
subplot(int(num_sims/4)+1,4,i+2)
imshow(SC_scaled)
colorbar()
title('SC')
tight_layout()

In [ ]:
PSD_show = True
plot_psd = [0,1,3]
if PSD_show:
    freqs = [0.5*i for i in range(4,100)]
    figure(figsize=(16,60))
    counter = 0
    for i in range(68):
        counter+=1
        subplot(17,4,counter)
        plot(freqs,psd[plot_psd[0],i,:96],label = f'Uncoupled',linestyle = '-')
        plot(freqs,psd[plot_psd[1],i,:96],label = f'Coupled {plot_psd[1]}',linestyle = '-')
        plot(freqs,psd[plot_psd[2],i,:96],label = f'Coupled {plot_psd[2]}',linestyle = '-')
        #xlim([1,40])
        legend(loc = 'upper right')
        title(f"Region: {i}")
        grid(linestyle = '--')
    tight_layout()
    show()

In [ ]:
ts_show = True
plot_ts = [0,1,3]
if ts_show:
    t = [5e-4*j for j in range(4000,5000)]
    figure(figsize=(16,15))
    for i in range(3):
        subplot(3,1,i+1)
        for j in range(68):
            plot(t,results[plot_ts[i]][0][j,4000:5000])
        title(f"Simulation: {plot_ts[i]}")
    
        grid(linestyle = '--')
    tight_layout()
    show()